In [29]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pathlib import Path
import nltk
from pygooglenews import GoogleNews

In [30]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [31]:
# Set the ticker
ticker = input()

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=past_date,
    end=current_date,
    after=None,
    until=None,
).df

# Display data
df.tail()

TSLA                                       
                              open       high      low    close    volume
time                                                                     
2021-11-19 00:00:00-05:00  1099.16  1138.7198  1092.70  1137.12  19639641
2021-11-22 00:00:00-05:00  1162.01  1201.9500  1132.43  1156.05  30542562
2021-11-23 00:00:00-05:00  1167.51  1180.4999  1062.70  1108.81  34397054
2021-11-24 00:00:00-05:00  1080.39  1132.7700  1062.00  1115.85  20960960
2021-11-26 00:00:00-05:00  1099.44  1108.7827  1081.00  1081.79  10205161

In [32]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2021-11-01,1209.4800
2021-11-02,1171.8500
2021-11-03,1213.6880
2021-11-04,1229.7300
2021-11-05,1221.4406


In [33]:
# Use the `pct_change` function to calculate daily returns of AAPL
stock_returns = df.pct_change().dropna()

# Display sample data
stock_returns.head()

,close
2021-11-02,-0.031113
2021-11-03,0.035703
2021-11-04,0.013218
2021-11-05,-0.006741
2021-11-08,-0.048067


In [34]:
stock=input()
stock_headlines = newsapi.get_everything(
    q=stock,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Stock article: {stock_headlines['totalResults']}")

# Show sample article
stock_headlines["articles"][0]


Total articles about Stock article: 8317


{'source': {'id': 'the-verge', 'name': 'The Verge'},
 'author': "Sean O'Kane",
 'title': 'JPMorgan says Tesla owes the bank $162 million',
 'description': 'JPMorgan has sued Tesla, and claims it’s owed $162 million after Tesla refused to fully settle a 2014 stock warrant agreement. The dispute centers on changes made to the agreement following Tesla CEO Elon Musk’s infamous 2018 “funding secured” tweet.',
 'url': 'https://www.theverge.com/2021/11/15/22784319/jpmorgan-tesla-lawsuit-162-million-stock-warrants-musk',
 'urlToImage': 'https://cdn.vox-cdn.com/thumbor/WvwRLzZgCTbUsTaqzjR-rIyxtmA=/0x146:2040x1214/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/19430216/acastro_191204_1777_elon_legal_0003.0.jpg',
 'publishedAt': '2021-11-16T02:15:38Z',
 'content': 'More fallout from the infamous funding secured tweet\r\nIllustration by Alex Castro / The Verge\r\nJPMorgan has sued Tesla, claiming that the electric car company owes the bank $162 million related to a … [+5127 chars]'}

In [35]:
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

In [36]:
# Instantiate SentimentIntensityAnalyzer
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aelri\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [37]:
keyword=input()
all_headlines, dates = get_headlines(keyword)

Fetching news about 'Tesla'
******************************
retrieving news from: 2021-11-29 00:00:00
retrieving news from: 2021-11-28 00:00:00
retrieving news from: 2021-11-27 00:00:00
retrieving news from: 2021-11-26 00:00:00
retrieving news from: 2021-11-25 00:00:00
retrieving news from: 2021-11-24 00:00:00
retrieving news from: 2021-11-23 00:00:00
retrieving news from: 2021-11-22 00:00:00
retrieving news from: 2021-11-21 00:00:00
retrieving news from: 2021-11-20 00:00:00
retrieving news from: 2021-11-19 00:00:00
retrieving news from: 2021-11-18 00:00:00
retrieving news from: 2021-11-17 00:00:00
retrieving news from: 2021-11-16 00:00:00
retrieving news from: 2021-11-15 00:00:00
retrieving news from: 2021-11-14 00:00:00
retrieving news from: 2021-11-13 00:00:00
retrieving news from: 2021-11-12 00:00:00
retrieving news from: 2021-11-11 00:00:00
retrieving news from: 2021-11-10 00:00:00
retrieving news from: 2021-11-09 00:00:00
retrieving news from: 2021-11-08 00:00:00
retrieving news f

In [38]:
stock_sentiment=[]
for article in all_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        stock_sentiment.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
stock_df = pd.DataFrame(stock_sentiment)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
stock_df = stock_df[cols].set_index('date')

filtered = stock_df.drop(columns=['compound'])
filtered['close']=stock_returns['close']


TypeError: list indices must be integers or slices, not str

In [ ]:
stock_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.00000
mean,0.114395,0.045040,0.019850,0.93507
std,0.293997,0.051117,0.047658,0.07050
min,-0.848100,0.000000,0.000000,0.70900
25%,0.000000,0.000000,0.000000,0.88900
50%,0.000000,0.021000,0.000000,0.94100
75%,0.345300,0.078000,0.000000,1.00000
max,0.690800,0.198000,0.290000,1.00000


In [ ]:
gn = GoogleNews(country = "US")

def get_news(search):
    stories=[]
    search=gn.search(search)
    newsitem= search['entries']
    for item in newsitem:
        story={'title' : item.title,
               'link':item.link
               }
    stories.append(story)
    return stories



In [ ]:
print(get_news(input()))

[{'title': "EXCLUSIVE Dutch forensic lab says it has decoded Tesla's driving data - Reuters", 'link': 'https://www.reuters.com/business/autos-transportation/dutch-forensic-lab-says-it-has-decoded-teslas-driving-data-2021-10-21/'}]


In [ ]:
filtered.head()

,text,positive,negative,neutral,close
date,,,,,
2021-11-16,More fallout from the infamous funding secured...,0.078,0.000,0.922,NaN
2021-11-01,"As promised earlier in the year\r\n, Tesla is ...",0.070,0.000,0.930,NaN
2021-11-13,Are you uncomfortable with the idea of Tesla l...,0.081,0.076,0.843,NaN
2021-11-14,Don't despair if you want to stream a show at ...,0.078,0.000,0.922,NaN
2021-11-19,Female employees at Teslas Fremont factory in ...,0.092,0.199,0.709,NaN


In [52]:
df=pd.DataFrame(all_headlines,dates)
df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
2021-11-29,The Wall Street Journal: Nissan plans to spend...,Nissan Unveils $18 Billion Long-Term Electric-...,Today Is Delivery Day For My Tesla Model Y!,Concerns Stream In Hours Before Receiving Our ...,Nissan to Spend $17.6 Billion on Battery-Power...,12 Top-Rated Mattresses That Make It Hard To G...,EV battery: China powers Asia in race to ramp ...,‘Succession’ Season 3 Episode 7 Recap: “Too Mu...,Nissan to spend $17.6 bn over 5 years to incre...,Where new AWS CEO Adam Selipsky plans to take ...,Nissan unveils ¥2 trillion long-term electric ...,'The perfect situation.' A look inside UC's na...,Tesla Model Y Upgrades to AMD Ryzen Chip in Ch...,None,None,None,None,None,None,None
2021-11-28,Elon Musk tells Tesla staff to cut cost of del...,Short selling is becoming less common — but th...,Will Rivian Become the Next Tesla?,Tesla to Start Production in Germany in Decemb...,Should Tesla Have Built Its European Gigaplant...,"QOTD: Should Tesla Buy An Automaker And If So,...",Forget Cathie Wood’s $500K Bitcoin call — the ...,Tesla car melted down into bust of Elon Musk,New Update Informs Tesla’s FSD Beta Testers Th...,Musk Says “No Thanks” To German Battery Factor...,The green future of big rigs is almost here,VC has a pivotal role to play in the climate f...,"MG Australia Gets It: If You Charge It, They W...",Xiaomi plans electric vehicle factory to produ...,Which Are Chris Harris’s Favorite Cars Of 2021?,"May Mobility Already Is Ahead, Even As ‘Tortoi...",Elon Musk tells Tesla (TSLA) employees to worr...,Elon Musk Tells Tesla Employees Not to ‘Sprint...,A Tiny Cryptocurrency Called Omicron Is Sudden...,Porsche Studio stores are coming to the U.S.
2021-11-27,Xiaomi's upcoming EV factory will make up to 3...,Here's a look back at a month of tweets from t...,Every auto startup wants to be the next Tesla....,Will Rivian’s electric vehicles end Detroit’s ...,There's an easy way to tax billionaires that w...,Worried about AI ethics? Worry about developer...,3 EV Charts That Will Leave You Speechless,Tesla Model Y In China Now Comes With AMD Ryze...,"Xiaomi Auto Plant Reportedly to Have 300,000-U...",2021 Holiday Gift Guide: Kevin’s picks,Will Rivian Become The Next Tesla?,Ford’s Mach-E Mustang GT Looks More Like A Pin...,China Fund Manager Reaping 109% Is a Die-Hard ...,Xiaomi to start making Tesla rival in Beijing,Hertz-Tesla Deal Signals Broad Shift to EVs fo...,Mercedes EQB Launches In Germany At Higher Pri...,Elon Musk wants you to stop relying upon crypt...,Electric Car FAQs: Do EVs All Use The Same Plug?,Charitable Giving Primer; Talking Turkey With ...,Links 11/27/27
2021-11-26,Tesla withdrew state funding application for G...,"The Zacks Analyst Blog Highlights: Tesla, Alph...","Zacks.com featured highlights include: Tesla, ...",Apple global battery development chief moves t...,"Tesla Announces $20,000 USD Carbon-Ceramic Bra...",Carmakers get inventive as global chip crisis ...,Why Tesla Stock Tumbled Today,Can Lucid Challenge Tesla's EV Dominance?,TECHART Promotes Safe Tuning With POLIZEI-Them...,A Closer Look At Tesla’s Supercharging Network...,The Lancia Delta Integrale Is Returning to Ral...,The 1982 Mercedes-Benz 450 SL Trans-Am Racer I...,'Big shock': Stocks plunge amid fears over new...,Washington Watch: Here are the hurdles for Bid...,"Tesla in Pennsylvania catches fire, flames spr...",Tesla May Cancel Orders For Customers Who Dela...,The Day My Tesla Motor Fell Down …,Elon Musk wants Dogecoin owners to ditch crypt...,A Tesla Car Has Been Melted Down To Sculpt Elo...,"Tesla To Add $20,000 Carbon Ceramic Brake Kit ..."
2021-11-25,What if Tesla made a compact electric microcar?,China to remain 'super market' into next year ...,Jamie Dimon will eat more crow after Thanksgiv...,Elon Musk Sold Another $1 Billion in Tesla Shares,"Caviar's made an $8,400 iPhone out of a 'real ...",The weirdest future cars at this month’s auto ...,Desktop bust and custom iPhone 13 Pro made fro